In [0]:
#Mount drive files
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Install Fast Ai
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai

In [0]:
#import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [0]:

print(os.getcwd())

/content


In [0]:
#get biopaper data 
trainCsvPath = Path('drive/My Drive/Year5/csc4006 Final Project/data/bioPaperExport_text_label.csv')

df = pd.read_csv(trainCsvPath)
df.head()

,text,label
0,Speaking is a sensorimotor behavior whose neur...,neuroscience
1,Resistance to proteasome inhibitors (PIs) is a...,cancer biology
2,Migration through constrictions can clearly ru...,cell biology
3,Peripheral nerve decoding algorithms form an i...,bioengineering
4,Mechanosensitive ion channels are membrane gat...,molecular biology


In [0]:
#check col, row sizes
df.shape

(29319, 2)

In [0]:
#check paperSubjects remove nan
categories = set(df['label'])
print(categories)

{'cancer biology', 'pathology', 'clinical trials', 'synthetic biology', 'zoology', 'epidemiology', 'evolutionary biology', 'developmental biology', 'bioengineering', 'biophysics', 'immunology', 'animal behavior and cognition', 'paleontology', 'neuroscience', 'bioinformatics', 'systems biology', 'plant biology', 'scientific communication and education', 'pharmacology and toxicology', 'genomics', 'physiology', 'biochemistry', 'cell biology', 'microbiology', 'ecology', 'molecular biology', 'genetics', 'unknown'}


In [0]:
#check full abstract
df.iloc[0][0]

"Speaking is a sensorimotor behavior whose neural basis difficult to study at the resolution of single neurons due to the scarcity of human intracortical measurements and the lack of animal models. We recorded from electrode arrays in the 'hand knob' area of motor cortex in people with tetraplegia. Neurons in this area, which have not previously been implicated in speech, modulated during speaking and during non-speaking movement of the tongue, lips, and jaw. This challenges whether the conventional model of a 'motor homunculus' division by major body regions extends to the single-neuron scale. Spoken words and syllables could be decoded from single trials, demonstrating the potential utility of intracortical recordings for brain-computer interfaces (BCIs) to restore speech. Two neural population dynamics features previously reported for arm movements were also present during speaking: a large initial condition-invariant signal, followed by rotatory dynamics. This suggests that common 

In [0]:
#remove stop words using nltk library
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords 
stopWords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
#to lowercase
df['text'] = df['text'].str.lower()
df.iloc[0][0]

"speaking is a sensorimotor behavior whose neural basis difficult to study at the resolution of single neurons due to the scarcity of human intracortical measurements and the lack of animal models. we recorded from electrode arrays in the 'hand knob' area of motor cortex in people with tetraplegia. neurons in this area, which have not previously been implicated in speech, modulated during speaking and during non-speaking movement of the tongue, lips, and jaw. this challenges whether the conventional model of a 'motor homunculus' division by major body regions extends to the single-neuron scale. spoken words and syllables could be decoded from single trials, demonstrating the potential utility of intracortical recordings for brain-computer interfaces (bcis) to restore speech. two neural population dynamics features previously reported for arm movements were also present during speaking: a large initial condition-invariant signal, followed by rotatory dynamics. this suggests that common 

In [0]:
#preprocess attempt 01, limit to alphanumeric + apostrophe
df['text'] = df['text'].str.replace("[^a-zA-Z]", " ")
df.iloc[0][0]

'speaking is a sensorimotor behavior whose neural basis difficult to study at the resolution of single neurons due to the scarcity of human intracortical measurements and the lack of animal models  we recorded from electrode arrays in the  hand knob  area of motor cortex in people with tetraplegia  neurons in this area  which have not previously been implicated in speech  modulated during speaking and during non speaking movement of the tongue  lips  and jaw  this challenges whether the conventional model of a  motor homunculus  division by major body regions extends to the single neuron scale  spoken words and syllables could be decoded from single trials  demonstrating the potential utility of intracortical recordings for brain computer interfaces  bcis  to restore speech  two neural population dynamics features previously reported for arm movements were also present during speaking  a large initial condition invariant signal  followed by rotatory dynamics  this suggests that common 

In [0]:
#tokenise
tk = []
for ab in df['text']:
  tk.append(ab.split())

#remove stopwords
noStopWords = []
for item in tk:
  item = [word for word in item if word not in stopWords]
  noStopWords.append(item)
tk = noStopWords
tokenised = pd.Series(tk)
tokenised.iloc[0]


['speaking',
 'sensorimotor',
 'behavior',
 'whose',
 'neural',
 'basis',
 'difficult',
 'study',
 'resolution',
 'single',
 'neurons',
 'due',
 'scarcity',
 'human',
 'intracortical',
 'measurements',
 'lack',
 'animal',
 'models',
 'recorded',
 'electrode',
 'arrays',
 'hand',
 'knob',
 'area',
 'motor',
 'cortex',
 'people',
 'tetraplegia',
 'neurons',
 'area',
 'previously',
 'implicated',
 'speech',
 'modulated',
 'speaking',
 'non',
 'speaking',
 'movement',
 'tongue',
 'lips',
 'jaw',
 'challenges',
 'whether',
 'conventional',
 'model',
 'motor',
 'homunculus',
 'division',
 'major',
 'body',
 'regions',
 'extends',
 'single',
 'neuron',
 'scale',
 'spoken',
 'words',
 'syllables',
 'could',
 'decoded',
 'single',
 'trials',
 'demonstrating',
 'potential',
 'utility',
 'intracortical',
 'recordings',
 'brain',
 'computer',
 'interfaces',
 'bcis',
 'restore',
 'speech',
 'two',
 'neural',
 'population',
 'dynamics',
 'features',
 'previously',
 'reported',
 'arm',
 'movements',


In [0]:

#detokenise
detokenised = [] 
for item in tk:
  newAbs = ' '.join(item)
  detokenised.append(newAbs)

df['text'] = detokenised

In [0]:
#split training and testing data (70:30)
from sklearn.model_selection import train_test_split
trainDf, testDf = train_test_split(df,stratify=df['label'], test_size = 0.33, random_state = 11)

trainDf.shape,testDf.shape

((19643, 2), (9676, 2))

In [0]:
trainDf['text'].unique().size, testDf['text'].unique().size

(19178, 9571)

In [0]:
#create models from data 
data_lm = TextLMDataBunch.from_df(train_df=trainDf,valid_df=testDf,path='',text_cols='text', label_cols='label')

data_clas = TextClasDataBunch.from_df(path='',train_df=trainDf, valid_df=testDf, vocab=data_lm.train_ds.vocab,text_cols='text', label_cols='label')

In [0]:
#Create Learner model from databunches
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103, drop_mult=0.7)

In [0]:
# train the learner object with learning rate = 1e-2
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,2.014740,1.524530,0.522737


In [0]:
#unfreeze the pre-trained part, and use “discriminative learning rate” to fine-tune the parameters.
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy
1,1.874605,1.427388,0.558599


In [0]:
#Save result
learn.save_encoder('ft_enc')

In [0]:
#Create classifier with fine tuned encoder
learn = text_classifier_learner(data_clas, drop_mult=0.7)
learn.load_encoder('ft_enc')

In [0]:
#Fit model
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,1.890603,1.445950,0.557462
